# Télécharger tous les fanarts favoris d'un compte sur wonaruto

Ce programme permet de récupérer les fanarts favoris d'un compte, y compris si l'auteur/trice avait supprimé son fanart.

Pour exécuter le code, modifier les cellules où "/!\ Cellule modifiable" apparaît au dessus (pour écrire l'URL de vos favoris, et le nom du dossier où vous voulez télécharger les fanarts). Si le nom du dossier n'existe pas dans votre drive, le dossier sera créé.

Ne touchez à rien d'autre :) !

Vous pouvez ensuite lancer le programme en cliquant sur l'onglet "Exécution > Tout exécuter". Il faudra alors donner l'autorisation d'utiliser drive.

Laisser l'onglet ouvert !!

============================================================================

# Cellules à modifier

Veuillez modifier toutes les cellules suivantes avec vos infos

/!\ Cellule modifiable

Mettre le lien de votre galerie wonaruto ici, entre les guillemets.

In [ ]:
URL_favs = "http://www.wonaruto.com/fanarts/favoris/270905/Rukia-Jericho/"

/!\ Cellule modifiable

On va nommer ici notre dossier où l'on veut que tous nos fanarts se trouvent. Le dossier n'a pas besoin d'être créé en amont.

In [ ]:
folder_name = "favoris_wonaruto"

# Code
Ne touchez à rien d'autre à partir d'ici :) ! Vous pouvez maintenant exécuter le programme en cliquant sur "Exécution > Tout exécuter"

============================================================================

Installation des frameworks nécessaires pour la suite

In [ ]:
!pip install bs4
!pip install requests

Import des frameworks nécessaires pour pouvoir les utiliser

In [ ]:
from bs4 import BeautifulSoup
import requests
from google.colab import drive

On va créer le dossier dans drive. Il faut pour cela donner l'autorisation d'utiliser drive lorsqu'on execute la cellule suivante.

In [ ]:
drive.mount('/content/drive/')
path_name = "drive/MyDrive/" + folder_name + "/"
!mkdir {path_name}

Création des variables...

In [ ]:
fanarts_links = []
max_page = 1
headers = {
    'Access-Control-Allow-Origin': '*',
    'Access-Control-Allow-Methods': 'GET',
    'Access-Control-Allow-Headers': 'Content-Type',
    'Access-Control-Max-Age': '3600',
    'User-Agent': 'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:52.0) Gecko/20100101 Firefox/52.0'
    }

Récupère le nombre de pages de favoris

In [ ]:
req = requests.get(URL_favs, headers)
soup = BeautifulSoup(req.content, 'html.parser')
nb_pages_div = soup.find("div", {"id": "pages_comment"}).find_all("a")

if len(nb_pages_div) >= 2:
  max_page = int(nb_pages_div[-2].get_text())

print("Il y a " + str(max_page) + " pages de favoris")

Récupère tous les liens des favoris

In [ ]:
for num_page in range(1,max_page):
  URL_fav_courant = URL_favs + str(num_page) + "/"

  req = requests.get(URL_fav_courant, headers)
  soup = BeautifulSoup(req.content, 'html.parser')

  all_arts = soup.find_all("div", {"class": "image_galerie"})

  for i in range(len(all_arts)):
    fanart_link = all_arts[i].find('a')
    if fanart_link != None:
      fanart_link = fanart_link['href']
      if len(fanart_link) >= 4 and fanart_link[:4] != "http":
        fanart_link = "http://www.wonaruto.com/" + fanart_link
      fanarts_links.append(fanart_link)


print("Il y a " + str(len(fanarts_links))+ " fanarts favoris")

Télécharge tous les fanarts favoris

In [ ]:
num_fanart = 0
for URL in fanarts_links:
  num_fanart += 1
  print("processing le fanart "+ str(num_fanart) + " des favoris...")

  req = requests.get(URL, headers)
  soup = BeautifulSoup(req.content, 'html.parser')

  table = soup.find_all('table')
  if len(table) >= 1:
    img = table[0].find('img')
    if img != None:
      fanart = img['src']

      if len(fanart) >= 4 and fanart[:4] != "http":
        fanart = "http://www.wonaruto.com/" + img['src']

      !wget -P {path_name} {fanart}